In [66]:
#1
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

df = pd.read_csv("ZIONO_weekly_return_volatility.csv")
df.label = df.label.fillna(method='ffill')
df_21 = df[df.Year==2021]
df_22 = df[df.Year==2022]

df_22.head(10)


df_22.head() # mean


,Year,Week_Number,mean_return,volatility,label
54,2022,1,-0.2012,0.842329,red
55,2022,2,0.1002,0.604448,green
56,2022,3,0.1525,0.591757,green
57,2022,4,-0.3576,1.847664,red
58,2022,5,-0.1206,0.762250,red


In [67]:
# 1 , construct df_train
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from scipy import stats

x_21 = df_21[["mean_return", "volatility"]].values
le = LabelEncoder()
y_21 = le.fit_transform(df_21["label"].values)
y_22 = le.transform(df_22["label"].values)


df_train = pd.DataFrame({'x1': df_21["mean_return"].values, 'x2': df_21['volatility'],  'y': y_21})
df_train.head()

,x1,x2,y
2,-0.2586,0.729467,0
3,0.0776,0.787634,1
4,0.9710,2.437100,0
5,-0.5042,1.114784,1
6,0.1458,1.217830,0


In [68]:
# !pip uninstall numpy 
# !where pip
# !where python
# !python -m pip install numpy==1.19 -i https://pypi.tuna.tsinghua.edu.cn/simple

    
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


X = df_train[[ 'x1', 'x2']]. values
scaler = StandardScaler (). fit (X)
X = scaler.transform (X)
Y = df_train['y']. values

In [69]:
lda_classifier = LDA( n_components =1)
lda_classifier .fit(X,Y)

new_x = scaler.transform(df_22[["mean_return", "volatility"]].values)
predicted = lda_classifier.predict ( new_x )
predicted

array([1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0])

In [70]:
y_22

array([1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1])

In [71]:
# 1 linear equation
lda_classifier.coef_, lda_classifier.intercept_

(array([[-2.06564214, -0.09743102]]), array([0.51791167]))

In [72]:
from sklearn . discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

qda_classifier = QDA (store_covariance=True)
qda_classifier .fit (X,Y)


QuadraticDiscriminantAnalysis(store_covariance=True)

In [73]:
qda_classifier.get_params()

{'priors': None, 'reg_param': 0.0, 'store_covariance': True, 'tol': 0.0001}

In [74]:
# 1. quadratic equation
print(qda_classifier.means_, qda_classifier.priors_, qda_classifier.scalings_ )
"covariance_ ", qda_classifier.covariance_ 

[[ 0.74398682  0.21928349]
 [-0.54559034 -0.16080789]] [0.42307692 0.57692308] [array([1.78335692, 0.68687659]), array([0.69178471, 0.31456082])]


('covariance_ ',
 [array([[0.98133137, 0.48596322],
         [0.48596322, 1.48890215]]),
  array([[ 0.35464194, -0.11624569],
         [-0.11624569,  0.65170359]])])

In [75]:
# 2 acc
l_predicted = lda_classifier.predict ( new_x )
q_predicted = qda_classifier.predict(new_x)
l_acc = np.mean(l_predicted == y_22)
q_acc = np.mean(q_predicted == y_22)
l_acc, q_acc


(0.7307692307692307, 0.5961538461538461)

In [76]:
# we can see that linear is better

In [77]:
# 3 linear
from sklearn.metrics import confusion_matrix
confusion_matrix(y_22, l_predicted)

array([[14, 13],
       [ 1, 24]], dtype=int64)

In [78]:
# 3 quadratic
confusion_matrix(y_22, q_predicted)

array([[ 8, 19],
       [ 2, 23]], dtype=int64)

In [79]:
# 4 linear  TFR TPR
tn, fp, fn, tp = confusion_matrix(y_22, l_predicted).ravel()
tnr = tn / (tn + fp)
tpr = tp / (tp + fn)
tnr, tpr

(0.5185185185185185, 0.96)

In [80]:
# 4 quadratic  TFR TPR
tn, fp, fn, tp = confusion_matrix(y_22, q_predicted).ravel()
tnr = tn / (tn + fp)
tpr = tp / (tp + fn)
tnr, tpr

(0.2962962962962963, 0.92)

In [81]:
# 5 linear profit
df_detail = pd.read_csv("ZIONO_weekly_return_detailed.csv")
df_detail.head(22)


df_detail_22 = df_detail[["Close", "Week_Number", "Year"]]
df_detail_22 = df_detail_22[df_detail_22.Year == 2022]
df_22_week = df_detail_22.groupby("Week_Number").last()
df_22_week.head()


,Close,Year
Week_Number,,
1,26.290001,2022
2,26.420000,2022
3,26.580000,2022
4,26.090000,2022
5,25.930000,2022


In [82]:
df_22_week.tail()

,Close,Year
Week_Number,,
48,25.180000,2022
49,25.129999,2022
50,25.299999,2022
51,25.120001,2022
52,25.370001,2022


In [83]:
# buy_and_hold
buy_hold_add = df_22_week.Close.values[-1] - df_22_week.Close.values[0]
buy_hold_add

-0.9200000762939453

In [84]:
# my strategy-- leaner
def get_benefit_my_strategy(pred_labels, close_prices):
    money = 100
    hold_share = False
    for i in range(len(pred_labels)): # 1 red
#         print(pred_labels[i], close_prices[i])
        
        if (pred_labels[i] == 1): # red---> if not owning, buy one
            if hold_share:
                pass
            else:
                hold_share = True
                money -= close_prices[i]
        else:           # green ---->if owning, sell it
            if hold_share:
                hold_share = False
                money += close_prices[i]
                
        
    share_value = close_prices[-1] if hold_share else 0
    return money + share_value - 100
                    

get_benefit_my_strategy( l_predicted.tolist(), df_22_week.Close.values.tolist())

1.3100013732910156

In [85]:
# my strategy-- quadratic
def get_benefit_my_strategy(pred_labels, close_prices):
    money = 100
    hold_share = False
    for i in range(len(pred_labels)): # 1 red
#         print(pred_labels[i], close_prices[i])
        
        if (pred_labels[i] == 1): # red---> if not owning, buy one
            if hold_share:
                pass
            else:
                hold_share = True
                money -= close_prices[i]
        else:           # green ---->if owning, sell it
            if hold_share:
                hold_share = False
                money += close_prices[i]
                
        
    share_value = close_prices[-1] if hold_share else 0
    return money + share_value - 100
                    

get_benefit_my_strategy( q_predicted.tolist(), df_22_week.Close.values.tolist())

0.0500030517578125

In [86]:
# we can see that
# linear > quadratic > buy_and_hold